In [53]:
#Import necessary libraries and APIs
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import getFamaFrenchFactors as gff
import seaborn as sns 

In [54]:
#Narrow search based on ticker symbol and start and end date of the historical data pull
ticker = 'AAPL'
start = '2017-11-17'
end = '2022-11-01'

In [55]:
#Download the stock data based on the variables outlined above
stock_data = yf.download(ticker, start, end, adjusted=True)
stock_data.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-11-17,42.759998,42.847500,42.410000,42.537498,40.448906,87598000
2017-11-20,42.572498,42.639999,42.389999,42.494999,40.408482,65049600
2017-11-21,42.695000,43.424999,42.695000,43.285000,41.159687,100525200
2017-11-22,43.340000,43.750000,43.262501,43.740002,41.592350,102355600
2017-11-24,43.775002,43.875000,43.662498,43.742500,41.594727,56106800


In [56]:
#Run the Fama French library to pull, Mkt-RF (market risk factor), SMB (performance of small cap stocks relative to large cap) and HML (high book to market value stocks compared to low book to market)
ff_monthly = gff.famaFrench3Factor(frequency='m')
ff_monthly.rename(columns={"date_ff_factors": 'Date'}, inplace=True)
ff_monthly.set_index('Date', inplace=True)
ff_monthly

,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07-31,0.0296,-0.0256,-0.0243,0.0022
1926-08-31,0.0264,-0.0117,0.0382,0.0025
1926-09-30,0.0036,-0.0140,0.0013,0.0023
1926-10-31,-0.0324,-0.0009,0.0070,0.0032
1926-11-30,0.0253,-0.0010,-0.0051,0.0031
...,...,...,...,...
2022-05-31,-0.0034,-0.0185,0.0841,0.0003
2022-06-30,-0.0843,0.0209,-0.0597,0.0006
2022-07-31,0.0957,0.0281,-0.0410,0.0008


In [57]:
#Looking just at the adjusted close amounts on a per month basis from the Yahoo Finance pull, look at the percent change in comparison to prior month's data, and drop empty data fields
stock_returns = stock_data['Adj Close'].resample('m').last().pct_change().dropna()

#Merge stock details with the Fama French details
stock_returns.name = "Month_Rtn"
ff_data = ff_monthly.merge(stock_returns,on='Date')
ff_data.tail()

,Mkt-RF,SMB,HML,RF,Month_Rtn
Date,,,,,
2022-05-31,-0.0034,-0.0185,0.0841,0.0003,-0.054496
2022-06-30,-0.0843,0.0209,-0.0597,0.0006,-0.081430
2022-07-31,0.0957,0.0281,-0.0410,0.0008,0.188634
2022-08-31,-0.0378,0.0139,0.0031,0.0019,-0.031208
2022-09-30,-0.0936,-0.0081,0.0005,0.0019,-0.120977


In [ ]:
#Define variables required to calculate the beta values
X = ff_data[['Mkt-RF', 'SMB', 'HML']]
y = ff_data['Month_Rtn'] - ff_data['RF']
X = sm.add_constant(X)

In [ ]:
#Develop a model using the variables defined above
ff_model = sm.OLS(y, X).fit()
print(ff_model.summary())
intercept, b1, b2, b3 = ff_model.params

In [ ]:
#Redefine values to help calculate the CAPM value
x_capm = ff_data[['Mkt-RF']]
y_capm = ff_data['Month_Rtn'] - ff_data['RF']
X_capm = sm.add_constant(x_capm)

In [ ]:
#Run the CAPM calculation to find the intercept, and corresponding betas
ff_model_capm = sm.OLS(y_capm, X_capm).fit()
print(ff_model_capm.summary())
intercept_capm, b1_capm = ff_model_capm.params

In [ ]:
intercept_capm

In [ ]:
b1_capm

In [ ]:
#Define variables to calculate the expected return using both the CAPM and Fama French
mkt_rf = .61
hml = .38
smb = .25
rf = .33

In [48]:
#Create and run the CAPM expected return calculation using the variables and inputs from above
capm_exreturn = rf + (mkt_rf*b1_capm)
calc_capm_exreturn = (1+(capm_exreturn/100))**12-1
calc_capm_exreturn

0.13544424624359142

In [50]:
# Print results
print(f"Using the CAPM model, our expected rate of return on {ticker} is {round(100*calc_capm_exreturn,2)}%.")

Using the CAPM model, our expected rate of return on AAPL is 13.54%.


In [ ]:
#Create and run the Fama French expected return calculation using the variables and inputs from above
ff_exreturn = rf + (mkt_rf*b1)+(smb*b2)+(hml*b3)
calc_ff_exreturn = (1+(ff_exreturn/100))**12-1
calc_ff_exreturn

In [52]:
# Print results
print(f"Using the Fama French 3 Factor model, our expected rate of return on {ticker} is {round(100*calc_ff_exreturn,2)}%.")

Using the Fama French 3 Factor model, our expected rate of return on AAPL is 10.48%.


In [ ]:
#Display a regression of the market return agains the chosen stock
sns.regplot(data = ff_data, x = 'Mkt-RF', y='Month_Rtn' )